In [ ]:
!pip install -q tensorflow Pillow matplotlib seaborn numpy pandas

In [ ]:
# !pip install -q cudf-cu11 --extra-index-url=https://pypi.nvidia.com

In [ ]:
import os
import re
import string
from pathlib import Path
from shutil import copyfileobj
from urllib.request import urlopen

# import cudf # cudf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import json
from random import shuffle
import pandas as pd
import datetime

import zipfile

print(f"TF version: {tf.__version__}")

In [ ]:
# %load_ext cudf.pandas

In [ ]:
def archive(dir: Path):
    with zipfile.ZipFile(f"{dir}.zip", "w", zipfile.ZIP_DEFLATED) as zip_file:
        for entry in dir.rglob("*"):
            zip_file.write(entry, entry.relative_to(dir))


def unarchive(file: Path):
    with zipfile.ZipFile(file, "r") as zip_file:
        zip_file.extractall(file.with_suffix(""))

In [ ]:
def download_file(url, dataset_file_path):
    path = Path(dataset_file_path)
    os.makedirs(path.parent, exist_ok=True)
    if not path.exists():
        print(f"Downloading {path}")
        with urlopen(url) as fsrc, open(path, "wb") as fdst:
            copyfileobj(fsrc, fdst)
    else:
        print(f"File {path} exists")

In [ ]:
dataset_path = "tmp/time_series_prediction"
batch_size = 2048
sequence_stride = 1

In [ ]:
download_file("https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip",
              f"{dataset_path}.zip")

In [ ]:
unarchive(Path(f"{dataset_path}.zip"))

In [ ]:
df = pd.read_csv(f"{dataset_path}/jena_climate_2009_2016.csv", delimiter=",", index_col=[0], dtype=str)

In [ ]:
df

In [ ]:
df.index = pd.to_datetime(df.index, format="%d.%m.%Y %H:%M:%S")

In [ ]:
df = df.apply(pd.to_numeric)

In [ ]:
def deduplicate(df):
  return df[~df.index.duplicated(keep="first")]

df = deduplicate(df)

In [ ]:
sample_period_s = df.index[1].timestamp() - df.index[0].timestamp()
sample_period_s

In [ ]:
def seconds_to_hms(seconds):
  import datetime
  from datetime import datetime as dt
  return str(datetime.timedelta(seconds=seconds))

In [ ]:
# Uneven data points
[(index, i, seconds_to_hms(diff_s)) for index, i, diff_s in zip(df.index, range(df.size), np.diff(df.index.map(lambda x: x.timestamp()))) if diff_s != sample_period_s]

In [ ]:
# os.makedirs(f"{dataset_path}/processed", exist_ok=True)
# df.to_csv(f"{dataset_path}/processed/jena_climate_2009_2016_processed.csv")
# archive(Path(f"{dataset_path}/processed"))

In [ ]:
hour_s = 60 * 60
day_s = 24 * hour_s
week_s = 7 * day_s
year_s = (365 * 3 + 366) / 4.0 * day_s
month_s = year_s / 12
year_months = year_s / month_s
year_weeks = year_s / week_s
year_days = year_s / day_s
year_hours = year_s / hour_s

In [ ]:
samples_per_year = year_s / sample_period_s
dataset_years = len(df) / samples_per_year
dataset_years

In [ ]:
def plot_df(df):
  colors = plt.rcParams["axes.prop_cycle"]()
  f, ax = plt.subplots(nrows=len(df.columns), figsize=(15, 15), sharex=True)
  for index, column in enumerate(df.columns):
    ax[index].plot(df.index, df[column], label=column, color=next(colors)["color"])
    ax[index].legend(loc="upper left")
  _ = plt.show()

plot_df(df)

In [ ]:
# def fix_column(column):
#   mean = column.mean()
#   threshold = mean * 100
#   return column.map(lambda x: x if abs(x) <= abs(threshold) else mean)

# for column in df.columns:
#   df[column] = fix_column(df[column])

In [ ]:
# def filter_df(df):
#   threshold = df.mean().abs() * 100
#   return df.apply(lambda row: row if all(row.abs() <= threshold) else 0, axis=1)

# df = filter_df(df)

# Faster version

def filter_df(df):
  threshold = df.mean().abs() * 100
  for index, row in df.iterrows():
    for column in df.columns:
      if abs(row[column]) > threshold[column]:
        df.loc[index] = 0

filter_df(df)

In [ ]:
df = df[~(df == 0).all(axis=1)]

In [ ]:
plot_df(df)

In [ ]:
def show_heatmap(data):
    plt.matshow(data.corr())
    plt.xticks(range(data.shape[1]), data.columns, fontsize=14, rotation=90)
    plt.gca().xaxis.tick_bottom()
    plt.yticks(range(data.shape[1]), data.columns, fontsize=14)

    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    plt.title("Feature Correlation Heatmap", fontsize=14)
    plt.show()


show_heatmap(df)

In [ ]:
# Features

In [ ]:
df["wd (deg) sin"] = np.sin(df["wd (deg)"] / 180 * np.pi)
df["wd (deg) cos"] = np.cos(df["wd (deg)"] / 180 * np.pi)
df = df.drop("wd (deg)", axis=1)
df["wd x"] = df["wv (m/s)"] * df["wd (deg) cos"]
df["wd y"] = df["wv (m/s)"] * df["wd (deg) sin"]

In [ ]:
fft = tf.signal.rfft(df["T (degC)"])
f_per_year = np.arange(0, len(fft)) / dataset_years
plt.step(x=f_per_year, y=np.abs(fft))
plt.xscale("log")
plt.xlim([0.1, max(plt.xlim())])
plt.xticks([1, year_months, year_weeks, year_days, year_hours],
           labels=["1/year", "1/month", "1/week", "1/day", "1/hour"])
plt.xlabel("Frequency (log scale)")
_ = plt

In [ ]:
df["day sin"] = np.sin(df.index.map(lambda x: x.timestamp()) * (2 * np.pi / day_s))
df["day cos"] = np.cos(df.index.map(lambda x: x.timestamp()) * (2 * np.pi / day_s))
df["week sin"] = np.sin(df.index.map(lambda x: x.timestamp()) * (2 * np.pi / week_s))
df["week cos"] = np.cos(df.index.map(lambda x: x.timestamp()) * (2 * np.pi / week_s))
df["year sin"] = np.sin(df.index.map(lambda x: x.timestamp()) * (2 * np.pi / year_s))
df["year cos"] = np.cos(df.index.map(lambda x: x.timestamp()) * (2 * np.pi / year_s))
date_features = ["day sin", "day cos", "week sin", "week cos", "year sin", "year cos"]
date_feature_count = len(date_features)

In [ ]:
plt.plot(np.array(df["day sin"])[:int(day_s / sample_period_s)])
plt.plot(np.array(df["day cos"])[:int(day_s / sample_period_s)])
plt.xlabel("Sample index")
plt.title("Time of day sin/cos")
_ = plt

In [ ]:
plt.plot(np.array(df["week sin"])[:int(week_s / sample_period_s)])
plt.plot(np.array(df["week cos"])[:int(week_s / sample_period_s)])
plt.xlabel("Sample index")
plt.title("Week sin/cos")
_ = plt

In [ ]:
split_size = 0.8
train_df = df[:int(len(df) * split_size)]
val_df = df[len(train_df):]

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()
df = (df - train_mean) / train_std

In [ ]:
start_ts = df.index[0]
end_ts = df.index[-1]

for ts in pd.date_range(start_ts, end_ts, freq=f"{sample_period_s}S"):
  if ts not in df.index:
    df.loc[ts] = 0 # A timestep is masked in TF only if all features are 0, so should be ok for cases when only a few features are 0

df = df.sort_index()

assert np.all(np.diff(df.index.map(lambda x: x.timestamp())) == sample_period_s)

In [ ]:
plot_df(df)

In [ ]:
train_df = df[:int(len(df) * split_size)]
val_df = df[len(train_df):]

In [ ]:
df_normalized = df.melt(var_name="Column", value_name="Normalized")
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x="Column", y="Normalized", data=df_normalized)
_ = ax.set_xticklabels(df.keys(), rotation=90)

In [ ]:
list(tf.keras.utils.timeseries_dataset_from_array(
    [1, 2, 3, 4, 5, 6, 7],
    targets=None,
    sequence_length=4,
    sequence_stride=3,
    batch_size=batch_size,
    shuffle=False,
).as_numpy_iterator())

In [ ]:
sequence_length = int(day_s * 5 / sample_period_s)
sequence_length

In [ ]:
prediction_length = int(day_s / sample_period_s)
prediction_length

In [ ]:
def make_dataset(df):
    return tf.keras.utils.timeseries_dataset_from_array(
        np.array(df, dtype=np.float16),
        targets=None,
        sequence_length=sequence_length + prediction_length,
        sequence_stride=sequence_stride,
        batch_size=batch_size,
        shuffle=False,
    ).map(lambda x: (x[:, :sequence_length], x[:, sequence_length:]))


# next(make_dataset(train_df).as_numpy_iterator())[1]

In [ ]:
train_df

In [ ]:
len(next(make_dataset(train_df).as_numpy_iterator())[1][0])

In [ ]:
feature_count = len(train_df.columns)
predicted_feature_count = feature_count - date_feature_count

In [ ]:
predicted_col_index = train_df.columns.get_loc("T (degC)")
# train_dataset = make_dataset(train_df).map(lambda seq, latest: (seq, latest[:, predicted_col_index]))
# val_dataset = make_dataset(val_df).map(lambda seq, latest: (seq, latest[:, predicted_col_index]))
train_dataset = make_dataset(train_df).map(lambda seq, pred: (seq, pred[:, :, :predicted_feature_count]))
val_dataset = make_dataset(val_df).map(lambda seq, pred: (seq, pred[:, :, :predicted_feature_count]))

In [ ]:
len(train_df)

In [ ]:
sum(len(batch[0]) for batch in train_dataset.as_numpy_iterator())

In [ ]:
train_df.columns

In [ ]:
next(train_dataset.as_numpy_iterator())[0].shape

In [ ]:
next(train_dataset.as_numpy_iterator())[1].shape

In [ ]:
(batch_size, prediction_length, predicted_feature_count)

In [ ]:
assert next(train_dataset.as_numpy_iterator())[0].shape == (batch_size, sequence_length, feature_count)
assert next(train_dataset.as_numpy_iterator())[1].shape == (batch_size, prediction_length, predicted_feature_count)

In [ ]:
# # Performance of naive prediction returning the previous value
# train_naive_pred_dataset = train_dataset.map(lambda seq, latest: (seq[:, -1, predicted_col_index], latest[:, predicted_col_index]))
# val_naive_pred_dataset = val_dataset.map(lambda seq, latest: (seq[:, -1, predicted_col_index], latest[:, predicted_col_index]))

In [ ]:
# np.array(next(train_naive_pred_dataset.as_numpy_iterator()))

In [ ]:
# model = tf.keras.Sequential([tf.keras.layers.Input((1,))])
# model.compile(loss="mse", metrics=["mae"])
# model.evaluate(train_naive_pred_dataset)
# model.evaluate(val_naive_pred_dataset)
# ""

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input((sequence_length, feature_count), dtype=np.float16),
    tf.keras.layers.Masking(mask_value=0),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(prediction_length * predicted_feature_count),
    tf.keras.layers.Reshape((prediction_length, predicted_feature_count))
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-3),
    loss="mse",
    metrics=["mae"]
)

In [ ]:
# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=f"tmp/checkpoints",
#     save_weights_only=False,
#     monitor=f"val_acc",
#     save_best_only=True,
# )
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor=f"val_loss",
)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=1e-4,
    patience=15,
    verbose=1,
    restore_best_weights=True,
    # restore_best_weights=False,
    start_from_epoch=0
)

In [ ]:
epochs = 5
history = model.fit(
    x=train_dataset.unbatch().shuffle(int(len(df) / 5)).batch(batch_size),
    # x=train_dataset.map(lambda x, y: (x[:, :, power_col_index], y)),
    epochs=epochs,
    # steps_per_epoch=250,
    callbacks=[reduce_lr_callback, early_stopping_callback],
    validation_data=val_dataset,
    # validation_data=val_dataset.map(lambda x, y: (x[:, :, power_col_index], y)),
    # validation_steps=20
)

In [ ]:
# LSTM only:                  loss: 3.3049 - mae: 0.7508 - val_loss: 4.1626 - val_mae: 1.0015 - lr: 1.0000e-04
# LSTM + Conv1D:              loss: 3.3213 - mae: 0.7442 - val_loss: 4.1755 - val_mae: 0.9871 - lr: 1.0000e-04
# LSTM without dates sin/cos: loss: 3.6518 - mae: 0.7578 - val_loss: 4.0217 - val_mae: 0.9691 - lr: 1.0000e-05
#

# sequence_length 1 week:
# LSTM + Conv1D:              Epoch 24/100 - 115s 349ms/step - loss: 5.3998e-04 - mae: 0.0155 - val_loss: 5.5193e-04 - val_mae: 0.0152 - lr: 0.0010

# sequence_length: 2 days
# LSTM + Conv1D:                   Epoch 20/20 - 34s  102ms/step - loss: 5.0597e-04 - mae: 0.0145 - val_loss: 5.4898e-04 - val_mae: 0.0151 - lr: 1.0000e-04
# LSTM:             too slow:      Epoch 3/20  - 318s 966ms/step - loss: 0.0017 -     mae: 0.0291 - val_loss: 0.0016 -     val_mae: 0.0270 - lr: 0.0010
# LSTM + Conv1D + Dense:           Epoch 20/20 - 35s  107ms/step - loss: 5.7357e-04 - mae: 0.0162 - val_loss: 0.0010 -     val_mae: 0.0229 - lr: 0.0010

# All features:
# LSTM + Conv1D:                                 34s 102ms/step -  loss: 0.0534 -     mae: 0.1067 - val_loss: 0.0576 -     val_mae: 0.1118 - lr: 0.0010
# LSTM + Conv1D 256:               Epoch 20/20 - 49s 149ms/step -  loss: 0.0530 -     mae: 0.1075 - val_loss: 0.0577 -     val_mae: 0.1122 - lr: 0.0010


In [ ]:
val_dataset_size = sum(len(batch[0]) for batch in val_dataset.as_numpy_iterator())

In [ ]:
val_dataset_size

In [ ]:
pred_steps = 1 * 10
input = next(val_dataset.as_numpy_iterator())[0][0]
input_s = val_df.index[0].timestamp()
predictions = []
for i in range(pred_steps):
  preds = model(tf.expand_dims(input, axis=0)).numpy()[0]
  for pred in preds:
    input_s += sample_period_s
    date_features = np.array([
        np.sin(input_s * 2 * np.pi / day_s),
        np.cos(input_s * 2 * np.pi / day_s),
        np.sin(input_s * 2 * np.pi / week_s),
        np.cos(input_s * 2 * np.pi / week_s),
        np.sin(input_s * 2 * np.pi / year_s),
        np.cos(input_s * 2 * np.pi / year_s),
    ])
    date_features = (date_features - train_mean[predicted_feature_count:]) / train_std[predicted_feature_count:]
    pred = np.concatenate([pred, date_features])
    predictions.append((input_s, pred))
    input = np.concatenate([input[1:], np.expand_dims(pred, axis=0)])

In [ ]:
pred_df = pd.DataFrame().reindex_like(df[:0])
pred_df = pd.concat([pred_df, pd.DataFrame([p for s, p in predictions], index=[pd.Timestamp(s, unit="s") for s, p in predictions], columns=df.columns)])

In [ ]:
actual_df = val_df[1:(1+pred_steps*prediction_length)]

In [ ]:
pred_df.shape

In [ ]:
actual_df.shape

In [ ]:
assert pred_df.shape == actual_df.shape

In [ ]:
mse = tf.keras.metrics.mean_squared_error(y_true=actual_df.transpose(), y_pred=pred_df.transpose())
mae = tf.keras.metrics.mean_absolute_error(y_true=actual_df.transpose(), y_pred=pred_df.transpose())

pd.DataFrame([mse.numpy()], columns=df.columns)

In [ ]:
# prediction_length=144, prediction_steps=1        =>  0.108
# prediction_length=12,  prediction_steps=12       =>  0.249
# prediction_length=12,  prediction_steps=120      =>  0.334
# prediction_length=144, prediction_steps=10 fp16  =>  0.405   or 0.355 for 3 epochs
# 144*10 fp16 batch_Size 2048   5 epochs   conv1d 128 + extra LSTM layer64        => 0.393793


In [ ]:
before_after_pred_df = pd.concat([train_df[-len(pred_df)*10:], val_df[0:1], pred_df])

In [ ]:
before_after_pred_df.shape

In [ ]:
plot_df(before_after_pred_df * train_std + train_mean)

In [ ]:
before_after_actual_df = pd.concat([train_df[-len(pred_df)*10:], val_df[0:1], val_df[1:(1+pred_steps*prediction_length)]])

In [ ]:
before_after_actual_df.shape

In [ ]:
assert before_after_pred_df.shape == before_after_actual_df.shape

In [ ]:
plot_df(before_after_actual_df * train_std + train_mean)

In [ ]:
diff_df = before_after_pred_df - before_after_actual_df

In [ ]:
plot_df(diff_df * train_std)